In [1]:
!python -V

Python 3.8.8


In [2]:
import pandas as pd

In [3]:
import pickle

In [4]:
import seaborn as sns
import matplotlib.pyplot as plt

In [5]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [9]:
import mlflow


mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

2023/06/01 21:34:59 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2023/06/01 21:34:59 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Running upgrade  -> 451aebb31d03, add metric step
INFO  [alembic.runtime.migration] Running upgrade 451aebb31d03 -> 90e64c465722, migrate user column to tags
INFO  [alembic.runtime.migration] Running upgrade 90e64c465722 -> 181f10493468, allow nulls for metric values
INFO  [alembic.runtime.migration] Running upgrade 181f10493468 -> df50e92ffc5e, Add Experiment Tags Table
INFO  [alembic.runtime.migration] Running upgrade df50e92ffc5e -> 7ac759974ad8, Update run tags with larger limit
INFO  [alembic.runtime.migration] Running upgrade 7ac759974ad8 -> 89d4b8295536, create latest metrics table
INFO  [89d4b8295536_create_latest_metrics_table_py] Migration complete!
INFO  

<Experiment: artifact_location='file:///C:/Users/91981/mlops-zoomcamp/02-experiment-tracking/mlruns/1', creation_time=1685635500876, experiment_id='1', last_update_time=1685635500876, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [8]:
pip install mlflow

  Using cached mlflow-2.3.2-py3-none-any.whl (17.7 MB)
  Using cached querystring_parser-1.2.4-py2.py3-none-any.whl (7.9 kB)
  Using cached databricks-cli-0.17.7.tar.gz (83 kB)
  Using cached waitress-2.1.2-py3-none-any.whl (57 kB)
  Using cached GitPython-3.1.31-py3-none-any.whl (184 kB)


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sphinx 4.0.1 requires Jinja2<3.0,>=2.3, but you have jinja2 3.1.2 which is incompatible.
sphinx 4.0.1 requires MarkupSafe<2.0, but you have markupsafe 2.1.1 which is incompatible.



  Using cached PyJWT-2.7.0-py3-none-any.whl (22 kB)
  Using cached gitdb-4.0.10-py3-none-any.whl (62 kB)
  Using cached smmap-5.0.0-py3-none-any.whl (24 kB)
  Created wheel for databricks-cli: filename=databricks_cli-0.17.7-py3-none-any.whl size=143878 sha256=4edc7c4286b795f88169994b96515c3bc7075262a81c6837cc09a1c3de2d39ec
  Stored in directory: c:\users\91981\appdata\local\pip\cache\wheels\65\c6\f0\85c95278b97ff8245c981b81f00bc05eb289cf79f884db2cbc
Successfully built databricks-cli
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.4
    Uninstalling urllib3-1.26.4:
      Successfully uninstalled urllib3-1.26.4
  Attempting uninstall: requests
    Found existing installation: requests 2.25.1
    Uninstalling requests-2.25.1:
      Successfully uninstalled requests-2.25.1
  Attempting uninstall: pywin32
    Found existing installation: pywin32 227
    Uninstalling pywin32-227:
      Successfully uninstalled pywin32-227


In [13]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [14]:
df_train = read_dataframe('C:/Users/91981/mlops-zoomcamp/data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('C:/Users/91981/mlops-zoomcamp/data/green_tripdata_2021-02.parquet')

In [15]:
len(df_train), len(df_val)

(73908, 61921)

In [16]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [17]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [18]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [19]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

7.479556169484162

In [21]:
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [22]:
with mlflow.start_run():

    mlflow.set_tag("developer", "mukesh")

    mlflow.log_param("train-data-path", "C:/Users/91981/mlops-zoomcamp/data/green_tripdata_2021-01.csv")
    mlflow.log_param("valid-data-path", "C:/Users/91981/mlops-zoomcamp/data/green_tripdata_2021-02.csv")

    alpha = 0.1
    mlflow.log_param("alpha", alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    mlflow.log_artifact(local_path="models/lin_reg.bin", artifact_path="models_pickle")

2023/06/01 21:38:35 WARNING mlflow.utils.git_utils: Failed to import Git (the Git executable is probably not on your PATH), so Git SHA is not available. Error: Failed to initialize: Bad git executable.
The git executable must be specified in one of the following ways:
    - be included in your $PATH
    - be set via $GIT_PYTHON_GIT_EXECUTABLE
    - explicitly set via git.refresh()

All git commands will error until this is rectified.

This initial warning can be silenced or aggravated in the future by setting the
$GIT_PYTHON_REFRESH environment variable. Use one of the following values:
    - quiet|q|silence|s|none|n|0: for no warning or exception
    - warn|w|warning|1: for a printed warning
    - error|e|raise|r|2: for a raised exception

Example:
    export GIT_PYTHON_REFRESH=quiet



In [25]:
pip install hyperopt

In [23]:
import xgboost as xgb

In [26]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [27]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [28]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [29]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

[21:39:49] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/objective/regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:9.37375                                                                                            
[1]	validation-rmse:7.14760                                                                                            
[2]	validation-rmse:6.75489                                                                                            
[3]	validation-rmse:6.66876                                                                                            
[4]	validation-rmse:6.64383                                                                                            
[5]	validation-rmse:6.62859                                                                                            
[6]	validation-rmse:6.62128                                                                            

[134]	validation-rmse:6.47776                                                                                          
[135]	validation-rmse:6.47760                                                                                          
[136]	validation-rmse:6.47727                                                                                          
[137]	validation-rmse:6.47810                                                                                          
[138]	validation-rmse:6.47648                                                                                          
[139]	validation-rmse:6.47574                                                                                          
[140]	validation-rmse:6.47598                                                                                          
[141]	validation-rmse:6.47724                                                                                          
[142]	validation-rmse:6.47901           

[33]	validation-rmse:6.54840                                                                                           
[34]	validation-rmse:6.54730                                                                                           
[35]	validation-rmse:6.54696                                                                                           
[36]	validation-rmse:6.54602                                                                                           
[37]	validation-rmse:6.54464                                                                                           
[38]	validation-rmse:6.54408                                                                                           
[39]	validation-rmse:6.54318                                                                                           
[40]	validation-rmse:6.54212                                                                                           
[41]	validation-rmse:6.54038            

[169]	validation-rmse:6.49795                                                                                          
[170]	validation-rmse:6.49756                                                                                          
[171]	validation-rmse:6.49747                                                                                          
[172]	validation-rmse:6.49740                                                                                          
[173]	validation-rmse:6.49743                                                                                          
[174]	validation-rmse:6.49720                                                                                          
[175]	validation-rmse:6.49712                                                                                          
[176]	validation-rmse:6.49713                                                                                          
[177]	validation-rmse:6.49704           

[45]	validation-rmse:6.67556                                                                                           
[46]	validation-rmse:6.67505                                                                                           
[47]	validation-rmse:6.67542                                                                                           
[48]	validation-rmse:6.67538                                                                                           
[49]	validation-rmse:6.67477                                                                                           
[50]	validation-rmse:6.67448                                                                                           
[51]	validation-rmse:6.67481                                                                                           
[52]	validation-rmse:6.67465                                                                                           
[53]	validation-rmse:6.67481            

...

[304]	validation-rmse:6.58880                                                                                          
[305]	validation-rmse:6.58861                                                                                          
[306]	validation-rmse:6.58841                                                                                          
[307]	validation-rmse:6.58807                                                                                          
[308]	validation-rmse:6.58772                                                                                          
[309]	validation-rmse:6.58754                                                                                          
[310]	validation-rmse:6.58732                                                                                          
[311]	validation-rmse:6.58705                                                                                          
[312]	validation-rmse:6.58698           

[440]	validation-rmse:6.56356                                                                                          
[441]	validation-rmse:6.56341                                                                                          
[442]	validation-rmse:6.56329                                                                                          
[443]	validation-rmse:6.56304                                                                                          
[444]	validation-rmse:6.56290                                                                                          
[445]	validation-rmse:6.56277                                                                                          
[446]	validation-rmse:6.56264                                                                                          
[447]	validation-rmse:6.56242                                                                                          
[448]	validation-rmse:6.56227           

[576]	validation-rmse:6.54287                                                                                          
[577]	validation-rmse:6.54288                                                                                          
[578]	validation-rmse:6.54277                                                                                          
[579]	validation-rmse:6.54256                                                                                          
[580]	validation-rmse:6.54242                                                                                          
[581]	validation-rmse:6.54230                                                                                          
[582]	validation-rmse:6.54214                                                                                          
[583]	validation-rmse:6.54204                                                                                          
[584]	validation-rmse:6.54188           

[712]	validation-rmse:6.52501                                                                                          
[713]	validation-rmse:6.52483                                                                                          
[714]	validation-rmse:6.52457                                                                                          
[715]	validation-rmse:6.52450                                                                                          
[716]	validation-rmse:6.52445                                                                                          
[717]	validation-rmse:6.52429                                                                                          
[718]	validation-rmse:6.52393                                                                                          
[719]	validation-rmse:6.52363                                                                                          
[720]	validation-rmse:6.52339           

KeyboardInterrupt: 

In [18]:
mlflow.xgboost.autolog(disable=True)

In [21]:
with mlflow.start_run():
    
    train = xgb.DMatrix(X_train, label=y_train)
    valid = xgb.DMatrix(X_val, label=y_val)

    best_params = {
        'learning_rate': 0.09585355369315604,
        'max_depth': 30,
        'min_child_weight': 1.060597050922164,
        'objective': 'reg:linear',
        'reg_alpha': 0.018060244040060163,
        'reg_lambda': 0.011658731377413597,
        'seed': 42
    }

    mlflow.log_params(best_params)

    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=1000,
        evals=[(valid, 'validation')],
        early_stopping_rounds=50
    )

    y_pred = booster.predict(valid)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")

[11:40:58] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-3.8/xgboost/src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:19.48425
[1]	validation-rmse:17.95634
[2]	validation-rmse:16.59114
[3]	validation-rmse:15.37412
[4]	validation-rmse:14.29011
[5]	validation-rmse:13.32800
[6]	validation-rmse:12.47570
[7]	validation-rmse:11.72140
[8]	validation-rmse:11.05888
[9]	validation-rmse:10.47583
[10]	validation-rmse:9.96483
[11]	validation-rmse:9.51784
[12]	validation-rmse:9.12620
[13]	validation-rmse:8.78522
[14]	validation-rmse:8.49037
[15]	validation-rmse:8.23332
[16]	validation-rmse:8.01179
[17]	validation-rmse:7.81822
[18]	validation-rmse:7.65209
[19]	validation-rmse:7.50743
[20]	validation-rmse:7.38235
[21]	validation-rmse:7.27382
[22]	validation-rmse:7.17865
[23]	validation-rmse:7.09606
[24]	validation-rmse:7.02481
[25]	validation-rmse:6.96336
[26]	validation-rmse:6.91001
[27]	valid

In [ ]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import LinearSVR

mlflow.sklearn.autolog()

for model_class in (RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, LinearSVR):

    with mlflow.start_run():

        mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.csv")
        mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.csv")
        mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

        mlmodel = model_class()
        mlmodel.fit(X_train, y_train)

        y_pred = mlmodel.predict(X_val)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)
        